In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Fungsi Syarat Batas
def bc(kodeBC, x, y, vx, vy, Lx, Ly, c):
    if kodeBC == 'bounceX':
        if x < 0:
            x = 0
            vx = -vx * c
        elif x > Lx:
            x = Lx
            vx = -vx * c

    elif kodeBC == 'bounceY':
        if y < 0:
            y = 0
            vy = -vy * c
        elif y > Ly:
            y = Ly
            vy = -vy * c

    elif kodeBC == 'periodicX':
        if x < 0:
            x += Lx
        elif x > Lx:
            x -= Lx

    elif kodeBC == 'periodicY':
        if y < 0:
            y += Ly
        elif y > Ly:
            y -= Ly

    return x, y, vx, vy

# Inisialisasi Variabel
x0, y0 = 0, 0     # Koordinat Awal
v, theta = 5, 60  # Kecepatan Awal (v) dan Sudut (theta)
gx, gy = 0, 9.8   # Konstanta Gravitasi pada sumbu-x dan sumbu-y
Lx, Ly = 10, 10   # Ukuran Box
c = 0.8           # Konstanta Redaman

vx = v * np.cos(np.deg2rad(theta))
vy = v * np.sin(np.deg2rad(theta))

dt = 0.1             # Time Step
Nlooping = int(2e2)  # Jumlah Looping

# Inisialisasi list untuk menyimpan hasil pergerakan
x_damped, y_damped = [], []
x_periodic, y_periodic = [], []

# Looping dengan Syarat Batas
x, y = x0, y0
for _ in range(Nlooping):
    # Hitung percepatan akibat gravitasi
    ax = -gx
    ay = -gy

    # Update kecepatan
    vx += ax * dt
    vy += ay * dt

    # Update posisi
    x += vx * dt
    y += vy * dt

    # Terapkan syarat batas untuk damped bounce
    x, y, vx, vy = bc('bounceX', x, y, vx, vy, Lx, Ly, c)
    x, y, vx, vy = bc('bounceY', x, y, vx, vy, Lx, Ly, c)

    # Simpan posisi ke dalam history untuk damped bounce
    x_damped.append(x)
    y_damped.append(y)

    # Terapkan syarat batas untuk periodic/wrap around
    x, y, vx, vy = bc('periodicX', x, y, vx, vy, Lx, Ly, c)
    x, y, vx, vy = bc('periodicY', x, y, vx, vy, Lx, Ly, c)

    # Simpan posisi ke dalam history untuk periodic/wrap around
    x_periodic.append(x)
    y_periodic.append(y)

# Inisialisasi plot
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot untuk syarat batas memantul dengan redaman
axs[0].set_xlim(0, Lx)
axs[0].set_ylim(0, Ly)
line_damped, = axs[0].plot([], [], 'bo-', lw=2)
line_pantul_damped, = axs[0].plot([], [], 'ro', lw=2)  # Tambahkan plot untuk pemantulan
axs[0].set_title('Damped Bounce')

# Plot untuk syarat batas periodic/wrap around
axs[1].set_xlim(0, Lx)
axs[1].set_ylim(0, Ly)
line_periodic, = axs[1].plot([], [], 'go-', lw=2)
axs[1].set_title('Periodic Wrap Around')

def init():
    line_damped.set_data([], [])
    line_periodic.set_data([], [])
    # Inisialisasi plot pemantulan
    line_pantul_damped.set_data([], [])
    return line_damped, line_periodic, line_pantul_damped

def update(frame):
    x_data_damped = x_damped[:frame]
    y_data_damped = y_damped[:frame]
    line_damped.set_data(x_data_damped, y_data_damped)

    x_data_periodic = x_periodic[:frame]
    y_data_periodic = y_periodic[:frame]
    line_periodic.set_data(x_data_periodic, y_data_periodic)

    # Update posisi partikel terpantul pada damped bounce
    x_pantul_damped = [x_damped[frame-1], x_damped[frame]] if frame > 0 else []
    y_pantul_damped = [y_damped[frame-1], y_damped[frame]] if frame > 0 else []
    line_pantul_damped.set_data(x_pantul_damped, y_pantul_damped)

    return line_damped, line_periodic, line_pantul_damped

# Buat animasi
ani = FuncAnimation(fig, update, frames=len(x_damped), init_func=init, blit=True, repeat=False)

# Tampilkan animasi
from IPython.display import HTML
HTML(ani.to_jshtml())
